In [ ]:
import json

In [ ]:
import os 
result_dir = 'logs/eval'
result_files = os.listdir(result_dir)
res = []
for file in result_files:
    if not file.endswith('.json'): continue
    with open(os.path.join(result_dir, file), 'r') as f:
        results = json.load(f)
    if any('categories' in v for round in results for k, v in round.items()):
        res.append((file, results))


In [ ]:
def get_success_rate(data):
    flat = [value for round in data for value in round.values()]

In [ ]:
r = json.load(open(result_dir + '/' + result_files[1]))

In [ ]:
list(repo for round in r for repo in round)

In [ ]:
repo_sets = [
    "eval/resources/python_repos_5k-1k.json",
    "eval/resources/python_repos_10k-5k.json",
    "eval/resources/python_repos_20k-10k.json",
    "eval/resources/python_repos_20k+.json"
]
repos = [json.load(open(repo_set, 'r')) for repo_set in repo_sets]
repos = [repo for repo_set in repos for repo in repo_set]

In [ ]:
repos_w_docs = list(filter(lambda x: len(x['relevant_docs']) > 0, repos))

In [ ]:
repos_w_docs

In [ ]:
from collections import Counter
doc_files = [repo['relevant_docs'] for repo in repos]
doc_files_clean = Counter([doc.split('/')[-1].split('.')[0].lower().replace('-', '_') for docs in doc_files for doc in docs])
doc_files = Counter([doc for docs in doc_files for doc in docs])


In [ ]:
print(f"total number of repos: {len(repos)}")
print(f"number of unique paths to relevant documentation: {len(doc_files)}")
print(f"number of unique names of relevant documentation: {len(doc_files_clean)}")
print(f"number of repos with no relevant documentation: {len([r for r in repos if len(r['relevant_docs']) == 0])}")

In [ ]:
from typing import List, Union
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, NullLocator
# import scienceplots
plt.style.use(['science'])

def bar(
    values: List[Union[float, int]],
    categories: List[str],
    x_label: str = "x",
    y_label: str = "y",
    title: str = "title",
):
    categories, values = zip(*sorted(zip(categories, values), key=lambda x: x[1] if x[0] != 'other' else -1))
    
    # remove minor ticks
    fig, ax = plt.subplots()
    ax.xaxis.set_minor_locator(NullLocator())
    ax.yaxis.set_minor_locator(NullLocator())
    
    plt.bar(categories, values)
    plt.xlabel(x_label)
    plt.xticks(rotation=90)
    plt.ylabel(y_label)
    plt.title(title)
    plt.tight_layout()
    plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))  # Force y-axis to use whole numbers
    plt.savefig(f"figs/{title.replace(' ', '_')}.pdf", format="pdf")
    plt.show()

In [ ]:
vals = {key: val for key, val in doc_files_clean.items() if val > 1}
vals['other'] = sum([v for v in doc_files_clean.values() if v <= 1])
bar(vals.values(), vals.keys(), x_label="Name of relevant document", y_label="Frequency", title="Frequency of Relevant Documentation Names")

In [ ]:
import os
dockerfiles_dir = 'resources/working_dockerfiles'
subdirs = os.listdir(dockerfiles_dir)
dockerfile_paths = []
dockerfiles = []
dockerfile_tags = [frozenset(repo['tags']) for repo in repos if 'tags' in repo]
df_filter = lambda x: (
    x != '' and not (x.startswith('#')
    or x.startswith('FROM') or x.startswith('WORKDIR') or x.startswith('COPY') )
)
for subdir in subdirs:
    for dockerfile in os.listdir(os.path.join(dockerfiles_dir, subdir)):
        df = os.path.join(dockerfiles_dir, subdir, dockerfile)
        if os.path.isfile(df):
            dockerfile_paths.append(df)
            dockerfiles.append(list(filter(df_filter, map(lambda x: x.strip(), open(df, 'r').readlines()))))
print(len(dockerfile_paths))

In [ ]:
flat_tags = set([tag for tag_set in dockerfile_tags for tag in tag_set])

In [ ]:
print(len(dockerfile_tags))
print(len(set(dockerfile_tags)))
print(len(flat_tags))

In [ ]:
from pprint import pprint
pprint(flat_tags)

In [ ]:
df_set = set('\n'.join(dockerfile) for dockerfile in dockerfiles)

In [ ]:
print(len(dockerfiles))
print(len(df_set))

In [ ]:
pprint(dockerfile_paths)